## Importing the necessary libraries

In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Bidirectional, SimpleRNN
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Loading the dataset

In [2]:
df = pd.read_csv("/kaggle/input/questionanalyserdataset/QuestionAnalyserDataset.csv")
df = df[['Questions','BloomsTaxClass']]
df

,Questions,BloomsTaxClass
0,Given is an array after the first partition of...,remember
1,How many steps are required to solve Tower of ...,remember
2,How many comparisons are required to find elem...,remember
3,"Given an array A[-3:4, 6:10], Find the address...",remember
4,"Consider the following list of 10 numbers: 35,...","apply , remember"
...,...,...
748,Find out the names of all American actors abov...,evaluate
749,Retrieve the name of each actor together with ...,apply
750,Retrieve details of all films that were releas...,apply
751,Find out the names of all actors that have pla...,evaluate


## Tokenizing the words in the Dataframe

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Questions'])
sequences = tokenizer.texts_to_sequences(df['Questions'])
max_len = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['BloomsTaxClass'])
vocab_size = len(tokenizer.word_index) + 1

## Spliting the data into training and testing sets

In [4]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)
print(type(X_train))

<class 'numpy.ndarray'>


## Defining the model architecture
Note that Here I have used the Bidirectional LSTM model *(As this topic is similar to Sentiment analysis)* 

In [5]:
inputs = Input(shape=(max_len,))
x = Embedding(vocab_size, 128, input_length=max_len)(inputs)
x = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2))(x)
outputs = Dense(50, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 203)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 203, 128)          306432    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               263168    
_________________________________________________________________
dense (Dense)                (None, 50)                12850     
Total params: 582,450
Trainable params: 582,450
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64)

Epoch 1/50
10/10 [==============================] - 22s 2s/step - loss: 3.6411 - accuracy: 0.1944 - val_loss: 2.7477 - val_accuracy: 0.1921
Epoch 2/50
10/10 [==============================] - 16s 2s/step - loss: 2.5764 - accuracy: 0.1844 - val_loss: 2.5076 - val_accuracy: 0.0927
Epoch 3/50
10/10 [==============================] - 17s 2s/step - loss: 2.4755 - accuracy: 0.2159 - val_loss: 2.4776 - val_accuracy: 0.1921
Epoch 4/50
10/10 [==============================] - 16s 2s/step - loss: 2.4410 - accuracy: 0.2326 - val_loss: 2.4348 - val_accuracy: 0.1921
Epoch 5/50
10/10 [==============================] - 17s 2s/step - loss: 2.3973 - accuracy: 0.2243 - val_loss: 2.4436 - val_accuracy: 0.1921
Epoch 6/50
10/10 [==============================] - 16s 2s/step - loss: 2.3286 - accuracy: 0.2708 - val_loss: 2.3572 - val_accuracy: 0.3046
Epoch 7/50
10/10 [==============================] - 17s 2s/step - loss: 2.2167 - accuracy: 0.3156 - val_loss: 2.2620 - val_accuracy: 0.2980
Epoch 8/50
10/10 [==

In [7]:
score, acc = model.evaluate(X_test, y_test, verbose=2)
print("Validation accuracy:", acc)

5/5 - 0s - loss: 3.2163 - accuracy: 0.3775
Validation accuracy: 0.37748345732688904


## Make predictions on new Data

In [8]:
new_questions = ['What types of programming languages are vulnerable to buffer overflows?', 'Construct the Binary Search Tree using following data. Show each steps. 32, 45, 12, 11, 13, 92, 78, 66, 17, 70,98, 108. Show its Preorder, Inorder and Postorder traversing sequences.']
new_sequences = tokenizer.texts_to_sequences(new_questions)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_len, padding='post')
predictions = model.predict(new_padded_sequences)
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))
print(predicted_labels)


['Remember' 'create']


# Implementation using Glove

In [9]:
# Import the necessary libraries
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Bidirectional
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [10]:
# Load the Bloom's Taxonomy dataset
#df = pd.read_csv("QuestionAnalyserDatasetUpdated.csv")
#df = df[['Questions','Blooms Taxonomy']]  

In [11]:
# Load the pre-trained GloVe embeddings
word_embeddings = {}
with open('/kaggle/input/glove6b100dtxt/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs

In [12]:
# Tokenize the text and convert it to sequences
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df['Questions'])
sequences = tokenizer.texts_to_sequences(df['Questions'])
max_len = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

In [13]:
# Create an embedding matrix for the pre-trained GloVe embeddings
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = word_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [14]:
# Convert the labels to numerical values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['BloomsTaxClass'])

In [15]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [37]:
# Define the model architecture
inputs = Input(shape=(max_len,))
x = Embedding(len(tokenizer.word_index) + 1, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)(inputs)
x = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2))(x)
outputs = Dense(50, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)
optimizer = keras.optimizers.Adam(learning_rate=0.005)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()


Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 203)]             0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 203, 100)          239400    
_________________________________________________________________
bidirectional_8 (Bidirection (None, 256)               234496    
_________________________________________________________________
dense_8 (Dense)              (None, 50)                12850     
Total params: 486,746
Trainable params: 247,346
Non-trainable params: 239,400
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64)

Epoch 1/50
10/10 [==============================] - 15s 1s/step - loss: 0.0833 - accuracy: 0.9635 - val_loss: 3.0336 - val_accuracy: 0.4172
Epoch 2/50
10/10 [==============================] - 16s 2s/step - loss: 0.0847 - accuracy: 0.9651 - val_loss: 2.9888 - val_accuracy: 0.3642
Epoch 3/50
10/10 [==============================] - 15s 1s/step - loss: 0.0963 - accuracy: 0.9635 - val_loss: 3.1971 - val_accuracy: 0.3775
Epoch 4/50
10/10 [==============================] - 15s 2s/step - loss: 0.0975 - accuracy: 0.9585 - val_loss: 3.0456 - val_accuracy: 0.3775
Epoch 5/50
10/10 [==============================] - 15s 2s/step - loss: 0.0970 - accuracy: 0.9585 - val_loss: 3.2325 - val_accuracy: 0.3907
Epoch 6/50
10/10 [==============================] - 15s 2s/step - loss: 0.0909 - accuracy: 0.9701 - val_loss: 2.9461 - val_accuracy: 0.3709
Epoch 7/50
10/10 [==============================] - 15s 1s/step - loss: 0.0944 - accuracy: 0.9668 - val_loss: 2.9180 - val_accuracy: 0.4106
Epoch 8/50
10/10 [==

In [36]:
score, acc = model.evaluate(X_test, y_test, verbose=2)
print("Validation accuracy:", acc)

5/5 - 0s - loss: 2.9013 - accuracy: 0.3974
Validation accuracy: 0.3973509967327118


In [19]:
# Make predictions on new data
new_questions = ['What can be the maximum number of nodes in binary tree with height 4?', 'Write an algorithm to insert a node at beginning in circular linked list.']
new_sequences = tokenizer.texts_to_sequences(new_questions)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_len, padding='post')
predictions = model.predict(new_padded_sequences)
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))
print(predicted_labels)

['remember' 'create']


## FastText Implementation Trial.

In [ ]:
import numpy as np
import pandas as pd
import fasttext
from tensorflow import keras
from keras.layers import Input, LSTM, Dense, Bidirectional, SimpleRNN
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# Load the data
df = pd.read_csv("/kaggle/input/questionanalyserdataset/QuestionAnalyserDataset.csv")
df = df[['Questions','BloomsTaxClass']]

In [ ]:
# Tokenize the questions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Questions'])
sequences = tokenizer.texts_to_sequences(df['Questions'])
max_len = max([len(seq) for seq in sequences])

In [ ]:
# Pad the sequences to a fixed length
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['BloomsTaxClass'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [ ]:
# Load the FastText pre-trained word embeddings
ft_model = fasttext.load_model('cc.en.300.bin')

# Generate word embeddings for each question in the dataset
embedding_dim = ft_model.get_dimension()
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_matrix[i] = ft_model[word]

In [ ]:
# Define the model architecture
inputs = Input(shape=(max_len,))
x = keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False)(inputs)
x = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2))(x)
outputs = Dense(50, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model..summary()

In [ ]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64)

In [ ]:
# Evaluate the model
score, acc = model.evaluate(X_test, y_test, verbose=2)
print("Validation accuracy:", acc)

In [ ]:
# Use the model to make predictions on new questions
new_questions = ['What types of programming languages are vulnerable to buffer overflows?', 'Construct the Binary Search Tree using following data. Show each steps. 32, 45, 12, 11, 13, 92, 78, 66, 17, 70,98, 108. Show its Preorder, Inorder and Postorder traversing sequences.']
new_sequences = tokenizer.texts_to_sequences(new_questions)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_len, padding='post')
predictions = model.predict(new_padded_sequences)
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))
print(predicted_labels)

In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Bidirectional, SimpleRNN
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold

# Load the data
df = pd.read_csv("QuestionAnalyserDataset.csv")
df = df[['Questions','Blooms Taxonomy']]

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Questions'])
sequences = tokenizer.texts_to_sequences(df['Questions'])
max_len = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')
vocab_size = len(tokenizer.word_index) + 1

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['Blooms Taxonomy'])

inputs = Input(shape=(max_len,))
x = Embedding(vocab_size, 128, input_length=max_len)(inputs)
x = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2))(x)
outputs = Dense(50, activation='sigmoid')(x)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

# Define k-fold cross validation
k = 5
kfold = KFold(n_splits=k, shuffle=True)

# Perform k-fold cross validation
test_losses = []
test_accuracies = []
for i, (train_idx, test_idx) in enumerate(kfold.split(padded_sequences)):
    X_train, X_test = padded_sequences[train_idx], padded_sequences[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    print('Fold:', i+1)
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64, verbose=1)

    # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)

# Print the average test loss and accuracy across all folds
print('Average test loss:', np.mean(test_losses))
print('Average test accuracy:', np.mean(test_accuracies))